# Transfer Learning: Sugarcane disease classification

This Repository consists of using Transfer Learning Method to detect types of Disease Sugarcane Plant can have.
Disease have following categories:
1. grassy shoot
2. leaf spot
3. nitrogen
4. orange rust
5. pyrilla
6. redrot
7. smut
8. wholly aphid
9. wilt
10. yellow leaf disease
11. leaf scald

In [1]:
!pip install keras

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import os
import shutil
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import log_loss, accuracy_score
%matplotlib inline

Using TensorFlow backend.


In [3]:
INPUT_SIZE=224
POOLING='avg'


#### Read Images, and Load them as Numpy array

In [5]:
from PIL import Image


def pil2numpy(img: Image = None) -> np.ndarray:
    """
    Convert an HxW pixels RGB Image into an HxWx3 numpy ndarray
    """

    if img is None:
        print ("None")
        img = Image.open('amsterdam_190x150.jpg')

    np_array = np.asarray(img)
    return np_array

def numpy2pil(np_array: np.ndarray) -> Image:
    """
    Convert an HxWx3 numpy array into an RGB Image
    """

    assert_msg = 'Input shall be a HxWx3 ndarray'
    assert isinstance(np_array, np.ndarray), assert_msg
    assert len(np_array.shape) == 3, assert_msg
    assert np_array.shape[2] == 3, assert_msg

    img = Image.fromarray(np_array, 'RGB')
    return img

def extract_features(path):
    directory_lists=os.listdir(path)
    X=[]
    Y=[]
    count=0
    plot_images = []
    if ('.DS_Store' in directory_lists):
            directory_lists.remove('.DS_Store')
    for d in directory_lists:
        nest=os.listdir(path+"/"+d)
        if ('.DS_Store' in nest):
            nest.remove('.DS_Store')
        print (d)
        print (len(nest))
        for f in nest:
            img = image.load_img(path+"/"+d+"/"+f, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = preprocess_input(img_data)
            X.append(img_data)
            Y.append(count)
#         plot_images.append(mpimg.imread(path+d+"/"+f))
        count+=1
    X=np.array(X)
    y=np.array(Y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    return X_train, X_test, y_train, y_test
# X_train, X_test, y_train, y_test, plot_images = extract_features("./data_2/")

# print (plot_images)





In [6]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# datagen = ImageDataGenerator(rotation_range=20, 
#                              zoom_range=0.15,
#                              width_shift_range=0.2,
#                              height_shift_range=0.2,
#                              shear_range=0.15,
#                              horizontal_flip=True,
#                              fill_mode="nearest")
# path = "./augmented/"
# directory_lists=os.listdir("./augmented/")
# for d in directory_lists:
#         nest=os.listdir(path+"/"+d)
#         if ('.DS_Store' in nest):
#             nest.remove('.DS_Store')
#         if len(nest)<20:
#             for f in nest:
#                 img = load_img(path+"/"+d+"/"+f)  
#                 x = img_to_array(img) 
#                 # Reshape the input image 
#                 x = x.reshape((1, ) + x.shape)  
#                 i = 0
#                 # generate 5 new augmented images 
#                 for batch in datagen.flow(x, batch_size = 1, 
#                                   save_to_dir = path+"/"+d+"/",  
#                                   save_prefix ='augmentation', save_format ='jpeg'):
#                     i += 1
#                     if i > 5: 
#                         break

#### We are using Transfer Learning using features extracted from VGG16, and Xception model trained on Image Net, we are just extracting BottleNeck Features, as we just want to learn generic features.

In [8]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import f1_score
def forward_pass_Resnt():
        X_train, X_test, y_train, y_test = extract_features("./no_aug_data/")
        print (X_train.shape)
        vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling=POOLING)
        train_vgg_bf = vgg_bottleneck.predict(X_train, batch_size=32, verbose=1)
        valid_vgg_bf = vgg_bottleneck.predict(X_test, batch_size=32, verbose=1)
        
#         xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
#         train_x_bf = xception_bottleneck.predict(X_train, batch_size=32, verbose=1)
#         valid_x_bf = xception_bottleneck.predict(X_test, batch_size=32, verbose=1)
        
#         X = np.hstack([train_x_bf, train_vgg_bf])
#         V = np.hstack([valid_x_bf, valid_vgg_bf])
# #         print (X.shape)
        logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=147)
        logreg.fit(train_vgg_bf, y_train)
        valid_probs = logreg.predict_proba(valid_vgg_bf)
        valid_preds = logreg.predict(valid_vgg_bf)
#         print ("validation probability: {}".format(valid_probs))
#         print ("validation_predictions: {}".format(valid_preds))
#         print ("Log Loss of Test Set:{}".format(log_loss(y_test, valid_probs)))
        print ("Accuracy:{}".format(accuracy_score(y_test, valid_preds)))
        print ("F1 Score",f1_score(y_test, valid_preds, average='macro'))
#         numpy_all = logreg.coef_
#         print (numpy_all)
#         clusterer = KMeans(n_clusters=11)
#         preds = clusterer.fit_predict(X)
#         centers = clusterer.cluster_centers_
#         score = silhouette_score(X, preds)
#         print ("Sillouette Score", score)

In [9]:
forward_pass_Resnt()

pyrilla
16
wholly_aphid
33
redrot
29
wilt
22
grassy_shoot
23
nitrogen
17
leaf_spot
4
leafscald
2
orange_rust
3
smut
4
yellow_leaf_disease
2
(108, 224, 224, 3)
47/47 [==============================] - 5s 117ms/step
Accuracy:0.574468085106383
F1 Score 0.39048359048359044


/opt/anaconda3/envs/project09/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
